In [4]:
# %pip install --upgrade google-meridian

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

Your runtime has 38.7 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


In [2]:
coord_to_columns = load.CoordToColumns(
    time='time',
    geo='geo',
    controls=['GQV', 'Competitor_Sales'],
    population='population',
    kpi='conversions',
    revenue_per_kpi='revenue_per_conversion',
    media=[
        'Channel0_impression',
        'Channel1_impression',
        'Channel2_impression',
        'Channel3_impression',
        'Channel4_impression',
    ],
    media_spend=[
        'Channel0_spend',
        'Channel1_spend',
        'Channel2_spend',
        'Channel3_spend',
        'Channel4_spend',
    ],
    organic_media=['Organic_channel0_impression'],
    non_media_treatments=['Promo'],
)

correct_media_to_channel = {
    'Channel0_impression': 'Channel_0',
    'Channel1_impression': 'Channel_1',
    'Channel2_impression': 'Channel_2',
    'Channel3_impression': 'Channel_3',
    'Channel4_impression': 'Channel_4',
}
correct_media_spend_to_channel = {
    'Channel0_spend': 'Channel_0',
    'Channel1_spend': 'Channel_1',
    'Channel2_spend': 'Channel_2',
    'Channel3_spend': 'Channel_3',
    'Channel4_spend': 'Channel_4',
}

In [3]:
loader = load.CsvDataLoader(
    csv_path="https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

/opt/anaconda3/envs/meridian/lib/python3.11/site-packages/meridian/data/load.py:1023: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df[geo_column_name] = self.df[geo_column_name].replace(
/opt/anaconda3/envs/meridian/lib/python3.11/site-packages/meridian/data/load.py:201: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if (constants.GEO) not in self.dataset.dims.keys():
/opt/anaconda3/envs/meridian/lib/python3.11/site-packages/meridian/data/load.py:231: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in fu

In [4]:
dir(data)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_unique_names',
 '_convert_geos_to_strings',
 '_validate_dimensions',
 '_validate_geos',
 '_validate_kpi',
 '_validate_media_channels',
 '_validate_names',
 '_validate_scenarios',
 '_validate_time',
 '_validate_time_coord_format',
 '_validate_time_formats',
 '_validate_times',
 'as_dataset',
 'control_variable',
 'controls',
 'frequency',
 'geo',
 'get_all_channels',
 'get_all_media_and_rf',
 'get_all_paid_channels',
 'get_n_top_largest_geos',
 'get_paid_channels_argument_builder',
 'get_paid_

In [5]:
data.kpi

<xarray.DataArray 'kpi' (geo: 40, time: 156)> Size: 50kB
array([[ 1954576.8,  2064249.6,  2086382.8, ...,  2052164.9,  2113785.8,
         2261657.5],
       [ 2126297.8,  1900217.6,  7177179. , ...,  3207906.5,  5641006. ,
          589451.4],
       [22316264. , 26041400. , 17542934. , ..., 16737179. , 18942866. ,
        26094610. ],
       ...,
       [14964052. , 15949083. , 16645703. , ..., 19246088. , 15721750. ,
        16629038. ],
       [ 4384791. ,  9319492. , 10441081. , ...,  5451277.5,  6433966.5,
         5571634.5],
       [ 9952539. ,  9143526. , 18018834. , ..., 10182924. , 12583538. ,
        10481814. ]])
Coordinates:
  * time     (time) object 1kB '2021-01-25' '2021-02-01' ... '2024-01-15'
  * geo      (geo) <U5 800B 'Geo0' 'Geo1' 'Geo10' ... 'Geo7' 'Geo8' 'Geo9'

In [6]:
data.media_spend

<xarray.DataArray 'media_spend' (geo: 40, time: 156, media_channel: 5)> Size: 250kB
array([[[2.0580608e+03, 0.0000000e+00, 0.0000000e+00, 3.6673965e+03,
         8.4160440e+02],
        [2.6852874e+03, 1.7557454e+03, 1.4731808e+02, 4.1122974e+03,
         1.9675044e+03],
        [1.4486895e+03, 2.2191223e+03, 0.0000000e+00, 3.0674023e+03,
         1.4341870e+03],
        ...,
        [3.4233550e+02, 0.0000000e+00, 0.0000000e+00, 4.6024440e+03,
         7.6995776e+02],
        [5.1081216e+02, 1.8144413e+03, 0.0000000e+00, 0.0000000e+00,
         2.1372435e+02],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.1945645e+03,
         0.0000000e+00]],

       [[2.1549700e+03, 1.8478578e+03, 8.0343160e+02, 3.2372627e+03,
         5.5359507e+03],
        [4.3814414e+03, 3.4804750e+03, 1.8959633e+03, 4.5576816e+03,
         3.5233933e+03],
        [7.4267285e+02, 4.4031410e+03, 0.0000000e+00, 2.7019724e+03,
         4.3548992e+01],
...
        [3.3285356e+03, 0.0000000e+00, 5.7644650e+02, 1.3448331e+04,
         9.7676910e+03],
        [0.0000000e+00, 7.3955283e+03, 0.0000000e+00, 7.8545600e+03,
         9.0035535e+02],
        [1.9417272e+03, 4.4512700e+03, 0.0000000e+00, 1.3831435e+04,
         4.9145380e+03]],

       [[2.7622734e+03, 1.2788032e+03, 0.0000000e+00, 1.1766535e+04,
         9.6404100e+03],
        [5.5479453e+03, 2.7744766e+03, 4.4975225e+03, 2.4147590e+04,
         8.7426570e+03],
        [6.3554985e+03, 9.5568710e+02, 0.0000000e+00, 1.7328395e+04,
         5.5778400e+03],
        ...,
        [0.0000000e+00, 0.0000000e+00, 1.5837753e+03, 2.2179390e+04,
         1.0209430e+04],
        [8.8670370e+03, 4.7934370e+03, 0.0000000e+00, 1.0030290e+04,
         7.5422354e+03],
        [5.9425635e+03, 1.2983365e+03, 6.7125176e+03, 2.1355562e+04,
         7.5620270e+03]]])
Coordinates:
  * time           (time) object 1kB '2021-01-25' '2021-02-01' ... '2024-01-15'
  * media_channel  (media_channel) <U9 180B 'Channel_0' ... 'Channel_4'
  * geo            (geo) <U5 800B 'Geo0' 'Geo1' 'Geo10' ... 'Geo7' 'Geo8' 'Geo9'

In [7]:
data.non_media_channel

<xarray.DataArray 'non_media_channel' (non_media_channel: 1)> Size: 8B
array(['Promo'], dtype=object)
Coordinates:
  * non_media_channel  (non_media_channel) object 8B 'Promo'

In [8]:
data.organic_media

<xarray.DataArray 'organic_media' (geo: 40, media_time: 156,
                                   organic_media_channel: 1)> Size: 50kB
array([[[  97320.],
        [ 201441.],
        [      0.],
        ...,
        [ 208967.],
        [      0.],
        [      0.]],

       [[      0.],
        [ 236553.],
        [      0.],
        ...,
        [ 575692.],
        [      0.],
        [  95607.]],

       [[2245629.],
        [      0.],
        [   7469.],
        ...,
...
        ...,
        [      0.],
        [      0.],
        [ 384866.]],

       [[ 328838.],
        [      0.],
        [ 163663.],
        ...,
        [1046102.],
        [ 398969.],
        [ 377178.]],

       [[1144514.],
        [ 517228.],
        [      0.],
        ...,
        [      0.],
        [ 709495.],
        [1167460.]]])
Coordinates:
  * media_time             (media_time) object 1kB '2021-01-25' ... '2024-01-15'
  * organic_media_channel  (organic_media_channel) object 8B 'Organic_channel...
  * geo                    (geo) <U5 800B 'Geo0' 'Geo1' ... 'Geo8' 'Geo9'

In [9]:
roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

I0000 00:00:1746692437.120364 10275272 service.cc:148] XLA service 0x1696b40d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746692437.120386 10275272 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1746692437.127213 10275272 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [10]:
mmm.sample_prior(500)

In [11]:
mmm.sample_posterior(n_chains=4, n_adapt=500, n_burnin=500, n_keep=1000, seed=1)

2025-05-08 11:21:54.068949: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1746692515.315785 10275272 assert_op.cc:38] Ignoring Assert operator mcmc_retry_init/assert_equal_1/Assert/AssertGuard/Assert


In [12]:
mmm